## Mini Project III

In [116]:
import numpy as np
import pandas as pd
import requests as re
import json

In [117]:
accounts = pd.read_csv('data/twm_accounts.csv', sep = ';')
accounts.name = 'accounts' 
checking_acct = pd.read_csv('data/twm_checking_acct.csv', sep = ';')
checking_acct.name = 'checking_acct'
checking_tran = pd.read_csv('data/twm_checking_tran.csv', sep = ';')
checking_tran.name = 'checking_tran'
credit_acct = pd.read_csv('data/twm_credit_acct.csv', sep = ';')
credit_acct.name = 'credit_acct'
credit_tran = pd.read_csv('data/twm_credit_tran.csv', sep = ';')
credit_tran.name = 'credit_tran'
customer = pd.read_csv('data/twm_customer.csv', sep = ';')
customer.name = 'customer'
savings_acct = pd.read_csv('data/twm_savings_acct.csv', sep = ';')
savings_acct.name = 'savings_acct'
savings_tran = pd.read_csv('data/twm_savings_tran.csv', sep = ';')
savings_tran.name = 'savings_tran'
transactions = pd.read_csv('data/twm_transactions.csv', sep = ';')
transactions.name = 'transactions'

In [118]:
CSVs = [accounts,
checking_acct,
checking_tran,
credit_acct,
credit_tran,
customer,
savings_acct,
savings_tran,
transactions]

# Data Preview

## acounts

In [119]:
accounts.head(1)

,acct_nbr,cust_id,acct_type,account_active,acct_start_date,acct_end_date,starting_balance,ending_balance
0,13628063,1362806,SV,Y,10.12.1995,NaN,1430.22,284.58


## checking acct

In [120]:
checking_acct.head(1)

,cust_id,acct_nbr,minimum_balance,per_check_fee,account_active,acct_start_date,acct_end_date,starting_balance,ending_balance
0,1362548,13625482,3000,0.0,Y,11.11.1986,NaN,6004.34,569.65


## checking tran

In [121]:
checking_tran.head(1)

,cust_id,tran_id,tran_amt,principal_amt,interest_amt,new_balance,tran_date,tran_time,channel,tran_code
0,1363481,26,-0.15,-0.15,0.0,58.99,29.1.1995,,,FK


## credit acct

In [122]:
credit_acct.head(1)

,cust_id,acct_nbr,credit_limit,credit_rating,account_active,acct_start_date,acct_end_date,starting_balance,ending_balance
0,1363160,4561143213631600,1000,0,Y,12.10.1988,NaN,657.46,286.69


## credit tran

In [123]:
credit_tran.head(1)

,cust_id,tran_id,tran_amt,principal_amt,interest_amt,new_balance,tran_date,tran_time,channel,tran_code
0,1363088,46,-121.49,-121.49,0.0,-141.0,20.10.1995,101144,E,CG


## customer

In [124]:
customer.head(1)

,cust_id,income,age,years_with_bank,nbr_children,gender,marital_status,name_prefix,first_name,last_name,street_nbr,street_name,postal_code,city_name,state_code
0,1362691,26150,46,5,1,M,2,,Donald ...,Marek ...,8298,Second ...,89194,Las Vegas,NV


## savings acct

In [125]:
savings_acct.head(1)

,cust_id,acct_nbr,minimum_balance,acct_type,account_active,acct_start_date,acct_end_date,starting_balance,ending_balance
0,1363160,13631603,100,BS,Y,9.5.1988,NaN,113.04,122.54


## savings tran

In [126]:
savings_tran.head(1)

,cust_id,tran_id,tran_amt,principal_amt,interest_amt,new_balance,tran_date,tran_time,channel,tran_code
0,1363481,26,136.03,136.03,0.0,1521.57,26.10.1995,84111,A,DP


## transactions

In [127]:
transactions.head(1)

,tran_id,acct_nbr,tran_amt,principal_amt,interest_amt,new_balance,tran_date,tran_time,channel,tran_code
0,27,13625623,0.0,0.0,0.0,3753.34,21.10.1995,121656,A,IQ


# Checking for Null Values and Outliers

In [128]:
customer.loc[:,['street_name','street_name','city_name','state_code']].isnull().any()

street_name    False
street_name    False
city_name      False
state_code     False
dtype: bool

In [129]:
customer.count()['cust_id']

747

In [130]:
savings_acct.count()['cust_id']

421

In [131]:
## They do not all have savings accounts

In [132]:
savings_acct[savings_acct['ending_balance'] == 0]

,cust_id,acct_nbr,minimum_balance,acct_type,account_active,acct_start_date,acct_end_date,starting_balance,ending_balance


In [133]:
##They all have money in it

In [137]:
customer_savings = savings_acct.loc[:,['cust_id','ending_balance']].reset_index(drop = True)

In [138]:
customer_savings

,cust_id,ending_balance
0,1363160,122.54
1,1362487,12.89
2,1362548,254.02
3,1362752,609.33
4,1363017,2901.35
...,...,...
416,1363141,528.57
417,1363038,508.26
418,1362529,873.14
419,1362895,96.25


In [134]:
credit_acct_without_empty = credit_acct.drop(credit_acct[credit_acct['ending_balance'] == 0].index.tolist())

In [135]:
customer_depth = credit_acct_without_empty.loc[:,['cust_id','ending_balance']].reset_index(drop = True)

In [136]:
customer_depth

,cust_id,ending_balance
0,1363160,286.69
1,1362487,1548.23
2,1362548,68.68
3,1363017,1000.00
4,1362609,3000.00
...,...,...
428,1363364,1785.97
429,1363467,1500.00
430,1363263,1218.94
431,1362569,9200.00
